<a href="https://colab.research.google.com/github/jikerbug/jibaek_project_generator/blob/master/Audio_Chord_Estimator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 개발 과정
1. 인공지능 알고리즘 개념 및 원리 학습
2. 코드 데이터 확보
3. 딥러닝 모델 생성
4. 딥러닝 모델 이용해 음악의 시간별 코드를 출력하는 프로그램 제작
5. 시간별 코드를 보여주는 GUI 제작

-

##**인공지능 모델 1차 개발 일지 ( 2020-07-03 ~ 2020-07-12 )**

### 1단계 : 인공지능 알고리즘 개념 및 원리 학습
1. 2020-07-03 ~ 2020-07-09 : [Deep Learning (for Audio) with Python ](https://www.youtube.com/playlist?list=PL-wATfeyAMNrtbkCNsLcpoAyBBRJZVlnf) 동영상 강의를 통해 신경망, CNN, RNN, LSTM 개념 학습과 음악 장르 분류 실습 진행
        오디오 신호에서 특성 값들을 추출해 신경망에 입력하여 음악장르를 예측하고 분류할 수 있다.


### 2단계 : 코드(chord) 데이터 확보
1. 2020-07-11 : [McGill-Billboard Songs and Chord Annotations
Chord Recognition with Chromagram Data](https://www.kaggle.com/jacobvs/mcgill-billboard)에서 데이터 확보 //주의! : code 설명을 보기전 데이터의 구조 파악 필수
        오디오 신호에서 추출한 Chromagram Data라는 24가지 특성 값을 통해 코드를 예측해보자

### 3단계 : 딥러닝 모델 생성 및 학습(code 설명)
1. 2020-07-11 ~ 2020-07-12 : 결과 : 48개의 코드 분류에 대해 약 31%의 정확도
       (Amaj, Amin, Abmaj, Abmin, A#maj, A#min, Bmaj .... G#min, None(코드 없는 audio)) : 총 48개

이상 1차 개발 내용은 아래의 코드에서 설명 : //주의! 설명이 다소 부정확할 수 있음




In [ ]:
import csv # 데이터를 csv 파일에서 불러오기 위함
import numpy as np # numpy array로 데이터를 변환하여 선형대수 연산을 하기 위함
from sklearn.model_selection import train_test_split # 모델을 학습시킬 training data와 모델의 예측 성능을 검증할 validation data, test data를 나눠주기 위함
import tensorflow.keras as keras # 인공지능 모델을 학습시키기 위함
import matplotlib.pyplot as plt # 학습된 결과를 그래플 시각화하기 위함
import os.path # 존재하지 않는 파일을 불러와서 오류가 나는 경우를 방지하기 위함 (by os.path.isfile(file_path))



# 1. 코드 데이터와 를 불러오는 함수
def load_data():

    #####목표 : 
    #####1. metadata 폴더에 있는 chromagram 데이터와 해당 chromagram의 시간값 확보 (인공지능 모델의 input : chromagram 데이터. ex : [1.214124, 2.141215, 0.1321, 0, 0, 2.21312, 1.21412 ....] )
    #####2. annotations 폴더에 있는 시간별 chord데이터를, chromagram의 시간값과 비교해 chromagram 데이터와 해당 chord값을 맵핑 (인공지능 모델의 output : chord 데이터. ex : 1(A:min이라는 뜻) )
    #####3. input과 output을 인공지능 모델에 맞게 가공
    
    ##1. 중복해서 사용하는 파일 경로를 저장
    path_metadata = './metadata/metadata'
    csv_metadata = '/bothchroma.csv'
    path_annotations = './annotations/annotations/'
    csv_annotations = '/majmin.lab'
    
    input_metadata_chroma_list =[] # csv 파일들에서 Chromagram Data를 전부 받아올 리스트
    output_annotations_chord_list =[] # lab 파일들(csv와 똑같이 취급 가능)에서 Chord Data를 전부 받아올 리스트
    chord_time_segmentation = [] # 특정 음악의 특정 코드의 시작과 끝 시간을 담을 리스트

    for folder_num in range(3,60): #데이터가 너무 많아서 일단은 0003 폴더 부터 0060 폴더 까지의 데이터로만 진행. 

        ##2. folder_num번째 폴더 파일 경로 저장
        path_variable = ''
        if(folder_num < 10):
            path_variable = '/000' + str(folder_num)
        elif(folder_num < 100):
            path_variable = '/00' + str(folder_num)
        elif(folder_num < 1000):
            path_variable = '/0' + str(folder_num)
        else:
            path_variable = '/' + str(folder_num)
        
        
        ##3. folder_num번째 폴더 파일 경로 저장()
        complete_path_metadata = path_metadata + path_variable + csv_metadata # chromagram data
        complete_path_annotations = path_annotations + path_variable + csv_annotations # chord data
        
        ##4. 파일이 존재하는지 체크
        if(os.path.isfile(complete_path_metadata)):
            pass
        else:
            continue #존재하지 않으면 아래에서 파일 open하는 과정 생략하고 위로 올라가서 for문 반복
        
        ##5. Chromagram Data를 불러와서 저장
        with open(complete_path_metadata, 'r', encoding='utf-8') as f:
            reader = csv.reader(f)
            for segmented_metadata in reader:
                input_metadata_chroma_list.append(segmented_metadata)

       ##6. Chord Data를 불러와서 저장
        with open(complete_path_annotations, 'r', encoding='utf-8') as f:
            reader = csv.reader(f)
            flag = 0
            for segmented_chord_data in reader:
                if(segmented_chord_data != []):
                    output_annotations_chord_list.append(segmented_chord_data[0].split('\t')[-1])
                    segmentation =[]
                    segmentation.append(segmented_chord_data[0].split('\t')[0])
                    segmentation.append(segmented_chord_data[0].split('\t')[1])

                    chord_time_segmentation.append(segmentation)
                else:
                    break
                #
                # flag +=1

    ##7. Chord Data를 0~41까지의 int형 변수로 저장 (7(코드) * 6(코드의 variation) = 42) + none(코드 없는 값) : 48로 설정
    #-> 48설정 이유 : 마지막 코드인 G#:min의 int변수 바로 다음인 42로 하기 보다는 결과값의 차이가 좀더 현저하게 나는 48로 설정해야 chord를 none으로 오판하는 일이 적을거라 생각)
    retyped_chord_to_int_list = [] #int로 변환된 코드들을 저장
    
    root_chord_list = ['A', 'B', 'C', 'D', 'E', 'F', 'G']

    type1 = ':maj'
    type2 = ':min'
    type3 = 'b:maj'
    type4 = 'b:min'
    type5 = '#:maj'
    type6 = '#:min'
    type_table = [type1, type2, type3, type4, type5, type6]

    for chord in output_annotations_chord_list:
        escape_flag = False
        for chord_num, root_chord in enumerate(root_chord_list):
            if (escape_flag):
                break
            for type_num, type in enumerate(type_table):
                if (chord == (root_chord + type)):
                    retyped_chord_to_int_list.append(chord_num * 4 + type_num)
                    escape_flag = True
                    break
                elif (chord == 'N' or chord == 'X'):
                    retyped_chord_to_int_list.append(48)
                    escape_flag = True
                    break


    ##8. input_metadata_chroma_list의 시간 데이터를 아래의 리스트로 옮기는 데이터 분류작업
    chroma_time_index_list = []
    for chroma in input_metadata_chroma_list:
        chroma_time_index_list.append(chroma[1])
        del chroma[0]
        del chroma[0]
        #len(chroma) == 24


    ##9. input_metadata_chroma_list의 string으로 되어있는 숫자값을 float으로 바꿔서 아래의 리스트에 넣어주기 (인공지능 모델에는 float 타입의 chromagram 데이터가 입력되어야 함!)
    retyped_chroma_string_to_float = []
    for chroma in input_metadata_chroma_list:
        retyped_list = []
        for value in chroma:
            temp_list = []
            temp_list.append(float(value))
            retyped_list.append(temp_list)
        retyped_chroma_string_to_float.append(retyped_list)


    # print(retyped_chroma_string_to_float[0])
    # print(chroma_time_index_list[0])
    # print(chord_time_segmentation[0])
    # print(retyped_chord_to_int_list[0])


    ##10. chromagram의 시간값(from chroma_time_index_list)이 위치하는 시간대의 chord정보(from chord_time_segmentation(시간대), retyped_chord_to_int_list(chord)))를 아래의 리스트에 저장
    chroma_time_related_chord_to_int_list = []
    for chroma_time in chroma_time_index_list:
        for time, chord in zip(chord_time_segmentation, retyped_chord_to_int_list):
            start = time[0]
            end = time[1]
            #print(f'chroma_time:{chroma_time} start:{start} end:{end}')
            if(float(chroma_time) >= float(start) and float(chroma_time) < float(end)):
                chroma_time_related_chord_to_int_list.append(chord)
                break


    # 데이터 구조 파악을 위한 출력함수
    # print("here")
    # print(len(chroma_time_related_chord_to_int_list))
    # print(len(retyped_chroma_string_to_float))
    # print(retyped_chroma_string_to_float[0])
    # print(chroma_time_related_chord_to_int_list[0])
    # for i in output_annotations_chord_list:
    #     print(i)

    # none 타입을 학습시키지 않기를 원하는 경우 사용할 코드
    #n_deleted_output_annotations_chord_list = output_annotations_chord_list
    # for chroma, chord in zip(input_metadata_chroma_list, output_annotations_chord_list):
    #     print("test" + chord)
        
    #     if chord == 'N':
            
    #         n_deleted_input_metadata_chroma_list.remove(chroma)
    #         n_deleted_output_annotations_chord_list.remove(chord)
    # print(len(retyped_chroma))
    # print(len(input_metadata_chroma_list))

    ##11. 인공지능 모델 학습을 위해 선형대수 연산이 가능하고 속도가 빠른 numpy array로 변환
    X = np.array(retyped_chroma_string_to_float)
    y = np.array(chroma_time_related_chord_to_int_list)


    return X, y




# 2. 인공지능 모델 학습결과를 시각화하는 함수
def plot_history(history):
    """Plots accuracy/loss for training/validation set as a function of the epochs
        :param history: Training history of model
        :return:
    """

    fig, axs = plt.subplots(2)

    # create accuracy sublpot
    axs[0].plot(history.history["accuracy"], label="train accuracy")
    axs[0].plot(history.history["val_accuracy"], label="test accuracy")
    axs[0].set_ylabel("Accuracy")
    axs[0].legend(loc="lower right")
    axs[0].set_title("Accuracy eval")

    # create error sublpot
    axs[1].plot(history.history["loss"], label="train error")
    axs[1].plot(history.history["val_loss"], label="test error")
    axs[1].set_ylabel("Error")
    axs[1].set_xlabel("Epoch")
    axs[1].legend(loc="upper right")
    axs[1].set_title("Error eval")

    plt.show()


# 3. 인공지능 모델에 입력할 데이터셋을 test, training, validation으로 구분하는 함수
def prepare_datasets(test_size, validation_size):


    # load data
    X, y= load_data()
    print("Look at here")
    print(len(X))
    print(len(y))

    # create train/test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size)

    # create train/validation split
    X_train, X_validation, y_train, y_validation = train_test_split(X_train, y_train, test_size=validation_size)

    # cnn과 달리 rnn에서는 이러한 3rd 차원이 필요 없다.
    # X_train = X_train[..., np.newaxis] # 3d array -> (num_samples = 130, 13, 1)
    # X_validation = X_validation[..., np.newaxis] # ... : 기존의 것들
    # X_test = X_test[..., np.newaxis]

    return X_train, X_validation, X_test, y_train, y_validation, y_test


# 4. 인공지능 모델을 구현하는 함수
def build_model(input_shape):

    # create RNN model
    model = keras.Sequential()

    # 2 LSTM layers
    model.add(keras.layers.LSTM(256, input_shape=input_shape, return_sequences=True)) #256개의 레이어나 64개의 레이어나 정확도는 약 31%로 똑같았다...
    # return_sequences : second lstm에서 이 시퀀스를 사용하고 싶기 떄문에 true로 한다.
    model.add(keras.layers.LSTM(256))

    # dense layer
    model.add(keras.layers.Dense(256, activation='relu'))
    model.add(keras.layers.Dropout(0.3))


    # output layer
    model.add(keras.layers.Dense(49, activation='softmax'))

    return model


# 5. 특정 데이터를 입력하여 결과값을 예측하는 함수
def predict(model, X, y):

    X = X[np.newaxis, ...]

    # prediction = [[0.1, 0.2, ...]] # softmax의 결과물
    prediction = model.predict(X) # X -> (1, 130, 13, 1)

    # extract index with max value
    predicted_index = np.argmax(prediction, axis=1) # [4]
    print("Expected index: {}, Predicted index: {}".format(y, predicted_index))

# 6. 메인 실행함수
if __name__ == "__main__":
    pass
    # create train, validation and test sets
    X_train, X_validation, X_test, y_train, y_validation, y_test = prepare_datasets(0.25, 0.2)

    # 데이터 구조 파악을 위한 print문
    # print(len(y_train))
    # print(y_train[0])
    # print(y_test[0])
    # print(y_validation[0])
    # print(len(X_train))
    # print(len(X_train[0]))
    # print(X_train[0])
    # print(X_test[0])
    # print(X_validation[0])
   
    # create network
    print("heoollo")
    print(X_train.shape[1],print(X_train.shape[1], ))
    input_shape = (X_train.shape[1], X_train.shape[2]) # (130, 13) (number of slices extract mfccs, mfccs)
    model = build_model(input_shape)

    # compile model
    optimizer = keras.optimizers.Adam(learning_rate=0.0001)
    model.compile(optimizer=optimizer,
                  loss="sparse_categorical_crossentropy",
                  metrics=['accuracy'])

    model.summary()

    # train model
    history = model.fit(X_train, y_train, validation_data=(X_validation, y_validation), batch_size=32, epochs=30)

    #plot accuracy/error for training and validation
    plot_history(history)

    # evaluate model on test set
    test_error, test_accuracy = model.evaluate(X_test, y_test, verbose=1)
    print("Accuracy on test set is: {}".format(test_accuracy))

    # make prediction on a sample
    X = X_test[100]
    y = y_test[100]


    predict(model, X, y)

